# Transformer training

In [1]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

In [2]:
#import statapp.transformer.common.get_positional_encodings
#from statapp.common.preprocessing import load_data, encode_data, split_into_X_y
def load_data(path, sample=1, split_on=" "):
    """Load a text dataset, optionally sampling the data rounding at some splitting string.
    
    Args:
        path (str): Path to a text file.
        sample (float): (approximate) Proportion of the dataset to load. 1 loads everything.
            Default: 1.
        split_on (str): String to split dataset on, e.g. "\n".
            When sampling, the dataset will be cut at the previous split_on substring.
            Ignored if sample==1.
            If None, samples the dataset by cutting at the previous character.
            Default: " ".
    
    Returns:
        string: Dataset.
    """
    with open(path, "r", encoding="utf-8") as file:
        text = file.read()
    
    if sample != 1:
        text = text[:int(len(text)*sample)]
        if split_on is not None:
            text = text[:-text[::-1].find(split_on)-1]
    
    return text

Preprocessing maison bien moche pour le moment... ^^

In [3]:
text = load_data("data/fr.train.top1M.txt")

In [4]:
tokens = nltk.word_tokenize(text[:1000000])

In [5]:
len(tokens)

193627

In [6]:
vocab = list(set(tokens))

In [7]:
vocab_size = len(vocab)

In [8]:
vocab_numbers = dict(zip(vocab, range(0,len(vocab))))

In [9]:
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

In [10]:
tokens_numbers_sequences = np.array([ tokens_numbers[i:i+vector_size] for i in range(len(tokens_numbers)-vector_size)])

In [11]:
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

In [12]:
nb_sequences =  tokens_numbers_sequences.shape[0]

In [13]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [17]:
def train_model(nb_epochs, batch_size):
    
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for batch_indices in batchs_indices:
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output[:][:-1], batch[:][1:])

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.

In [62]:
layernorm = nn.modules.normalization.LayerNorm(10)

In [63]:
a = torch.randn(1,1,5,10)

In [64]:
a

tensor([[[[-0.7605,  0.1726,  0.0760, -0.3939,  0.1223, -1.1342, -0.0594,
           -0.0268,  0.8039,  0.1352],
          [ 1.4146, -0.8825,  1.2408,  0.3367, -0.7527,  0.5585,  0.0327,
           -0.4742, -1.1078, -0.2232],
          [-0.9012, -0.8320, -0.5289,  1.5045,  0.5900, -1.2084,  0.5183,
            0.1394, -0.2172,  1.0590],
          [ 0.9643, -0.4161,  0.4669,  0.6355,  0.9358, -1.7287,  1.0459,
            1.1640,  1.0520,  0.2117],
          [-1.3917, -0.6418,  0.9964,  1.6676,  0.6557,  0.7827, -1.5622,
           -0.9920, -1.0246, -0.2748]]]])

In [65]:
layernorm(a)

tensor([[[[-1.2762,  0.5447,  0.3561, -0.5609,  0.4465, -2.0056,  0.0918,
            0.1555,  1.7766,  0.4716],
          [ 1.6966, -1.0866,  1.4861,  0.3906, -0.9292,  0.6594,  0.0223,
           -0.5919, -1.3595, -0.2878],
          [-1.0684, -0.9875, -0.6330,  1.7451,  0.6756, -1.4276,  0.5918,
            0.1486, -0.2684,  1.2240],
          [ 0.6211, -0.9931,  0.0395,  0.2366,  0.5878, -2.5279,  0.7165,
            0.8546,  0.7237, -0.2589],
          [-1.1365, -0.4340,  1.1005,  1.7293,  0.7814,  0.9003, -1.2962,
           -0.7621, -0.7926, -0.0902]]]], grad_fn=<NativeLayerNormBackward>)

In [68]:
layernorm(torch.randn(2,10))

tensor([[-0.1996, -0.6775,  0.5567,  1.3806, -0.2786,  0.6136, -2.3539, -0.0847,
         -0.0860,  1.1293],
        [ 0.7750, -1.2661,  0.1302, -1.5784,  1.2545, -0.6483, -0.0823,  1.7919,
         -0.1367, -0.2398]], grad_fn=<NativeLayerNormBackward>)

In [43]:
nn.Linear(10,10)(layernorm(torch.randn(2,10)))

tensor([[ 0.3994, -0.1170, -0.8294,  0.1222,  0.2536,  0.8992,  0.2525,  0.2596,
         -0.1398,  1.3155],
        [-0.4371, -0.0822, -1.2442, -0.2776,  1.0341,  1.5680, -0.6851,  0.0718,
         -0.1017, -0.3920]], grad_fn=<AddmmBackward>)

In [74]:
train_model(1,1)

C:\Users\Eric\statapp_language_model\statapp\transformer\pytorch\transformer_model.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(embedded + pos_encodings, dtype=torch.float32)


tensor([[[-0.0939,  1.2670,  0.0666,  ...,  0.7409,  0.4314, -0.1872],
         [-0.6540,  0.2155,  0.1585,  ...,  0.4980, -0.1013,  0.4925],
         [-0.4538,  0.3790, -0.0809,  ...,  1.2687,  0.4160,  0.6289],
         ...,
         [ 0.2204,  0.6733, -1.2449,  ...,  1.4116, -1.3340,  0.0312],
         [-1.4619,  1.3733,  1.2769,  ...,  1.2823, -0.0190,  0.4124],
         [-1.6102,  1.0765, -1.1784,  ...,  0.6709, -0.3816, -0.8799]]],
       grad_fn=<NativeLayerNormBackward>)
torch.Size([1, 512, 512])


TypeError: attention_mask() takes 1 positional argument but 2 were given